In [50]:
import pandas as pd
import sys, os
sys.path.append(os.path.abspath("../src"))

import dataanalyse
import importlib
importlib.reload(dataanalyse)


<module 'dataanalyse' from 'c:\\Users\\johan\\Documents\\Mappe\\src\\dataanalyse.py'>

In [ ]:
#Temperaturdata
statistikk_temp = dataanalyse.analyser_fil("../data/temp_gloshaugen_historisk.csv",datokolonne="tidspunkt",groupby="måned")

for navn, df in statistikk_temp.items():
    print(f"\nStatistikk: {navn}")
    display(df.round(2))


Analyse av:../data/temp_gloshaugen_historisk.csv
-------------------------------------------------

Statistikk: temperatur_måned


,år,måned,mean,median,std
0,2024,1,-5.26,-5.00,6.80
1,2024,2,-0.80,0.40,4.61
2,2024,3,2.66,3.00,2.78
3,2024,4,5.63,5.80,4.02
4,2024,5,15.96,15.60,4.55
5,2024,6,15.07,14.70,4.14
6,2024,7,16.78,16.25,3.30
7,2024,8,16.26,16.10,2.92
8,2024,9,13.26,13.00,4.39
9,2024,10,8.54,8.80,2.95



Statistikk: temperatur_år


,år,mean,median,std
0,2024,7.74,7.65,8.42


In [ ]:
#Klimasaddutslipp
statistikk_klimagass = dataanalyse.analyser_fil("../data/klimagassutslipp_renset.csv",sep=",",datokolonne="år",groupby="år")

for navn, df in statistikk_klimagass.items():
    print(f"\nStatistikk: {navn}")
    display(df.round(2))



Analyse av:../data/klimagassutslipp_renset.csv
-------------------------------------------------

Statistikk: utslipp til luft (1 000 tonn co2-ekvivalenter, ar5)_år


,år,mean,median,std
0,1990,11410.67,5298.0,15910.65
1,1991,10885.89,5109.0,15120.88
2,1992,10544.44,5461.0,14515.41
3,1993,10973.11,5664.0,15140.90
4,1994,11407.00,5858.0,15777.98
5,1995,11498.00,6554.0,15834.17
6,1996,12142.56,6960.0,16700.55
7,1997,12137.44,7246.0,16703.36
8,1998,12154.44,7463.0,16749.45
9,1999,12377.22,7526.0,17029.17
